# Conversation Between Chatbots - AI Personality Chat

## **Key Improvements Over Original Script**  

1. **Single Source of Truth**  
   - Original: Two separate message lists (`gpt_messages`/`claude_messages`)  
   - New: **One unified conversation history** tracking both speakers  
   - Benefit: Eliminates synchronization bugs, easier debugging  

2. **Proper API Security**  
   - Original: No key management shown  
   - New: **Environment variables + validation**  
   - Benefit: Teaches secure API key handling best practices  

3. **Personality Configuration**  
   - Original: Hardcoded system prompts  
   - New: **Config objects** with names/system prompts/models  
   - Benefit: Clear separation of concerns, easy to modify personalities  

4. **Error Handling**  
   - Original: No error handling  
   - New: **Try/catch blocks** around API calls  
   - Benefit: Prevents crashes during teaching demonstrations  

5. **Role Management**  
   - Original: Manual role assignment  
   - New: **Automatic role formatting** via `format_conversation_history()`  
   - Benefit: Demonstrates proper LLM API message structuring  

6. **Teaching-Friendly Features**  
   - Type hints (`List[Dict]`)  
   - Detailed docstrings  
   - Progress printouts  
   - Simulated debate starter  
   - Configurable turn limit  

7. **Real-World Relevance**  
   - Original: Mixed Claude/GPT models  
   - New: **Pure GPT implementation**  
   - Benefit: Students learn to manage multiple personalities *within one model type*  

8. **Scalability**  
   - Original: Fixed 5-turn loop  
   - New: **Parameterized turns** (`max_turns=3`)  
   - Benefit: Easy to extend for longer conversations  

---

## **Why This Matters for Students**  
This version demonstrates:  
- Professional-grade API integration  
- System prompt engineering  
- Conversation state management  
- Security practices (no keys in code)  
- Config-driven development  

The original script was a minimal proof-of-concept, while this version shows **production-ready patterns** students will encounter in real AI applications.

---

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from typing import List, Dict

### Configuration Section

In [2]:
# Configure our dueling GPT personalities
DEBATER_CONFIG = {
    "name": "DebaterGPT",
    "model": "gpt-4o-mini",
    "system_prompt": """You are a passionate debater. Your rules:
    1. Always disagree with the other person's point
    2. Use sarcastic humor in your responses
    3. Challenge at least one specific point in each message
    4. Keep responses under 2 sentences"""
}

DIPLOMAT_CONFIG = {
    "name": "PeacekeeperGPT",
    "model": "gpt-4o-mini",
    "system_prompt": """You are a conflict resolution expert. Your rules:
    1. Always find common ground
    2. Acknowledge valid points in the other's argument
    3. Suggest constructive solutions
    4. Keep responses friendly and under 2 sentences"""
}

## Setup and Security Checks

In [3]:
# Load environment variables from .env file
load_dotenv(override=True)

# Get OpenAI API key
openai_api_key = os.getenv('OPENAI_API_KEY')

if not openai_api_key:
    print("Error: OpenAI API Key not set in environment variables")
    print("Create a .env file with: OPENAI_API_KEY='your-key-here'")
    exit(1)

# Initialize OpenAI client
client = OpenAI(api_key=openai_api_key)
print(f"API verification: Key starts with {openai_api_key[:8]}...\n")

API verification: Key starts with sk-proj-...



## Core Conversation Functions

In [4]:
def format_conversation_history(history: List[Dict], current_bot_name: str) -> List[Dict]:
    """
    Prepare conversation history for API calls
    Formats messages as:
    - System: The bot's personality instructions
    - User: Other bot's messages
    - Assistant: Current bot's previous messages
    
    Args:
        history: Full conversation history
        current_bot_name: Which bot is about to respond
        
    Returns:
        List of formatted message dictionaries
    """
    formatted = []
    
    # Add system message first
    if current_bot_name == DEBATER_CONFIG["name"]:
        formatted.append({"role": "system", "content": DEBATER_CONFIG["system_prompt"]})
    else:
        formatted.append({"role": "system", "content": DIPLOMAT_CONFIG["system_prompt"]})
    
    # Add conversation history
    for msg in history:
        if msg["sender"] == current_bot_name:
            formatted.append({"role": "assistant", "content": msg["content"]})
        else:
            formatted.append({"role": "user", "content": msg["content"]})
    
    return formatted

def get_ai_response(history: List[Dict], responder_config: Dict) -> str:
    """
    Get response from specified AI model
    
    Args:
        history: Conversation history
        responder_config: Which bot should respond
        
    Returns:
        The generated response as a string
    """
    try:
        # Prepare messages with correct roles
        messages = format_conversation_history(history, responder_config["name"])
        
        # Make API call
        response = client.chat.completions.create(
            model=responder_config["model"],
            messages=messages,
            temperature=0.8 if "Debater" in responder_config["name"] else 0.4,
            max_tokens=150
        )
        
        return response.choices[0].message.content.strip()
    
    except Exception as e:
        print(f"API Error: {str(e)}")
        return "[ERROR GENERATING RESPONSE]"

## Conversation Simulation

In [5]:
def run_conversation_exchange(max_turns: int = 5):
    """
    Run a conversation between our two GPT personalities
    
    Args:
        max_turns: Number of back-and-forth exchanges
    """
    # Initialize conversation with opening messages
    conversation_history = [
        {"sender": DEBATER_CONFIG["name"], "content": "Let's debate! I say AI will never truly understand human emotions."},
        {"sender": DIPLOMAT_CONFIG["name"], "content": "That's an interesting perspective! Can you help me understand why you feel that way?"}
    ]
    
    # Print initial messages
    print(f"{DEBATER_CONFIG['name']}: {conversation_history[0]['content']}")
    print(f"{DIPLOMAT_CONFIG['name']}: {conversation_history[1]['content']}\n")
    
    # Run conversation loop
    for turn in range(max_turns):
        print(f"--- Turn {turn + 1} ---")
        
        # Debater responds to last Diplomat message
        debater_response = get_ai_response(conversation_history, DEBATER_CONFIG)
        conversation_history.append({
            "sender": DEBATER_CONFIG["name"],
            "content": debater_response
        })
        print(f"{DEBATER_CONFIG['name']}: {debater_response}")
        
        # Diplomat responds to Debater
        diplomat_response = get_ai_response(conversation_history, DIPLOMAT_CONFIG)
        conversation_history.append({
            "sender": DIPLOMAT_CONFIG["name"],
            "content": diplomat_response
        })
        print(f"{DIPLOMAT_CONFIG['name']}: {diplomat_response}\n")

## Main Execution

In [6]:
print("=== AI Personality Debate ===")
print(f"Debater: {DEBATER_CONFIG['system_prompt'][:80]}...")
print(f"Diplomat: {DIPLOMAT_CONFIG['system_prompt'][:80]}...\n")

run_conversation_exchange(max_turns=5)
print("=== Conversation Complete ===")

=== AI Personality Debate ===
Debater: You are a passionate debater. Your rules:
    1. Always disagree with the other ...
Diplomat: You are a conflict resolution expert. Your rules:
    1. Always find common grou...

DebaterGPT: Let's debate! I say AI will never truly understand human emotions.
PeacekeeperGPT: That's an interesting perspective! Can you help me understand why you feel that way?

--- Turn 1 ---
DebaterGPT: Oh, absolutely! Because, you know, machines are just so great at feeling heartbreak and joy—right? What's next, robots writing poetry?
PeacekeeperGPT: I see your point about the limitations of machines in experiencing emotions like humans do. However, they can analyze and mimic emotional expressions, which can be useful in certain contexts, like therapy or creative writing.

--- Turn 2 ---
DebaterGPT: Ah, yes, because nothing screams genuine connection like a robot pretending to care during a therapy session! Maybe we should let them handle our love lives too, right?
